In [1]:
import pandas as pd

In [2]:
import csv
csv_col=[]
with open("data/input/ppp_loans_state_CO.csv") as fp:
    csv_reader=csv.reader(fp,delimiter=",")
    csv_col=next(iter(csv_reader))

In [3]:
import json
content=None
json_col=[]
with open("data/input/places.json") as fp:
    content=json.loads(fp.read())
    json_col = list(next(iter(content['data'])).keys())


In [4]:
common_values=list(set(csv_col).intersection(set(json_col)))
print(common_values)

['address']


In [5]:
places_df_audit_city=pd.DataFrame(content['data'])
loans_df_gov=pd.read_csv("data/input/ppp_loans_state_CO.csv")

In [6]:
loans_df_gov.columns,len(loans_df_gov.columns)

(Index(['loan_amount', 'business_name', 'address', 'city', 'state', 'zip',
        'naics_code', 'business_type', 'race_ethnicity', 'gender', 'veteran',
        'non_profit', 'jobs_reported', 'date_approved', 'lender', 'cd',
        'source_file', 'na_flag', 'dupe_flag', 'year_approved', 'address_clean',
        'state_clean', 'city_clean'],
       dtype='object'),
 23)

In [7]:
def get_st_road_data(address):
    if isinstance(address,dict):
        return address.get('street',None)
    return None
def get_city(address):
    if isinstance(address, dict):
        return address.get('city',None)
    return None
def get_state(address):
    if isinstance(address, dict):
        return address.get('adm1',None)
    return None
def get_zipcode(address):
    if isinstance(address, dict):
        return address.get('postcode',None)
    return None

In [8]:
places_df_audit_city["address"]
places_df_audit_city["address_clean"]=places_df_audit_city['address'].apply(get_st_road_data)
places_df_audit_city["city_clean"]=places_df_audit_city['address'].apply(get_city)
places_df_audit_city["state_clean"]=places_df_audit_city['address'].apply(get_state)
places_df_audit_city["zip"]=places_df_audit_city['address'].apply(get_zipcode)
places_df_audit_city["business_name"]=places_df_audit_city['name']

In [9]:
places_df_audit_city[['business_name','address_clean','city_clean','state_clean','zip']][:6]

,business_name,address_clean,city_clean,state_clean,zip
0,CBDermaceuticals,None,Denver,Colorado,80202
1,Unique Manufacturing & Marketing,5752 N Lamar St,Arvada,Colorado,80002
2,S N N Manufacturing Inc,None,Denver,Colorado,80209
3,Monarch Metal Manufacturing,550 W 53rd Pl,Denver,Colorado,80216
4,T & H Manufacturing LLC,581 S Federal Blvd,Denver,Colorado,80219
5,Sana Packaging,3451 Decatur St,Denver,Colorado,80211


In [10]:
import fuzzymatcher

In [11]:
on=["business_name","address_clean","city_clean","state_clean","zip"]

In [12]:
matched_results = fuzzymatcher.fuzzy_left_join(places_df_audit_city,
                                            loans_df_gov,
                                            on,
                                            on,
                                            )

In [33]:
final_result=matched_results[matched_results["best_match_score"]>0.8]

In [34]:
matched_results.columns

Index(['best_match_score', '__id_left', '__id_right', 'rank', 'name', 'logo',
       'description', 'address_left', 'location', 'reviews', 'category',
       'timezone', 'price_range', 'language', 'is_claimed', 'extra',
       'identity', 'opening_hours', 'status', 'media', 'activity', 'devices',
       'external_id', 'feature_id', 'gkg_id', 'owner', 'phone', 'url',
       'appointment', 'website', 'address_clean_left', 'city_clean_left',
       'state_clean_left', 'zip_left', 'business_name_left', 'loan_amount',
       'business_name_right', 'address_right', 'city', 'state', 'zip_right',
       'naics_code', 'business_type', 'race_ethnicity', 'gender', 'veteran',
       'non_profit', 'jobs_reported', 'date_approved', 'lender', 'cd',
       'source_file', 'na_flag', 'dupe_flag', 'year_approved',
       'address_clean_right', 'state_clean_right', 'city_clean_right'],
      dtype='object')

In [35]:
final_result=final_result.sort_values(by=['best_match_score'],ascending=False)

In [36]:
basic_results=final_result[["business_name_left","business_name_right","best_match_score"]]

In [37]:
basic_results.to_csv("data/extra/business_names.csv")

In [38]:
final_result.to_csv('data/output/matched_result.csv',index=True)

In [39]:
df1_cols=list(places_df_audit_city.columns)
on.append("address")
for i in range(len(df1_cols)):
    if df1_cols[i] in on:
        df1_cols[i]=f'{df1_cols[i]}_left'

In [40]:
len(df1_cols), len(places_df_audit_city.columns), df1_cols

(32,
 32,
 ['rank',
  'name',
  'logo',
  'description',
  'address_left',
  'location',
  'reviews',
  'category',
  'timezone',
  'price_range',
  'language',
  'is_claimed',
  'extra',
  'identity',
  'opening_hours',
  'status',
  'media',
  'activity',
  'devices',
  'external_id',
  'feature_id',
  'gkg_id',
  'owner',
  'phone',
  'url',
  'appointment',
  'website',
  'address_clean_left',
  'city_clean_left',
  'state_clean_left',
  'zip_left',
  'business_name_left'])

In [41]:
matched_df_1=final_result[df1_cols]

In [42]:
df2_cols=['loan_amount',
       'business_name_right', 'address_right', 'city', 'state', 'zip_right',
       'naics_code', 'business_type', 'race_ethnicity', 'gender', 'veteran',
       'non_profit', 'jobs_reported', 'date_approved', 'lender', 'cd',
       'source_file', 'na_flag', 'dupe_flag', 'year_approved',
       'address_clean_right', 'state_clean_right', 'city_clean_right']


In [43]:
len(loans_df_gov.columns),len(df2_cols)

(23, 23)

In [44]:
matched_df_2=final_result[df2_cols]

In [45]:
matched_df_1.columns

Index(['rank', 'name', 'logo', 'description', 'address_left', 'location',
       'reviews', 'category', 'timezone', 'price_range', 'language',
       'is_claimed', 'extra', 'identity', 'opening_hours', 'status', 'media',
       'activity', 'devices', 'external_id', 'feature_id', 'gkg_id', 'owner',
       'phone', 'url', 'appointment', 'website', 'address_clean_left',
       'city_clean_left', 'state_clean_left', 'zip_left',
       'business_name_left'],
      dtype='object')

In [46]:
refined_matched_df_1=matched_df_1[['rank', 'name', 'logo', 'description', 'address_left', 'location',
       'reviews', 'category', 'timezone', 'price_range', 'language',
       'is_claimed', 'extra', 'identity', 'opening_hours', 'status', 'media',
       'activity', 'devices', 'external_id', 'feature_id', 'gkg_id', 'owner',
       'phone', 'url', 'appointment', 'website',"address_clean_left","state_clean_left","city_clean_left","zip_left"]]

In [47]:
refined_matched_df_1.rename(columns={
    "name":"business_name",
    "address_left":"address",
    "address_clean_left":"address_clean",
    "state_clean_left":"state_clean",
    "city_clean_left":"city_clean",
    "zip_left":"zip"
},inplace=True)

/tmp/ipykernel_10522/2393469631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  refined_matched_df_1.rename(columns={


In [48]:
matched_df_2.columns

Index(['loan_amount', 'business_name_right', 'address_right', 'city', 'state',
       'zip_right', 'naics_code', 'business_type', 'race_ethnicity', 'gender',
       'veteran', 'non_profit', 'jobs_reported', 'date_approved', 'lender',
       'cd', 'source_file', 'na_flag', 'dupe_flag', 'year_approved',
       'address_clean_right', 'state_clean_right', 'city_clean_right'],
      dtype='object')

In [49]:
matched_df_2.rename(columns={
    "business_name_right":"business_name",
    "address_right":"address",
    "address_clean_right":"address_clean",
    "state_clean_right":"state_clean",
    "city_clean_right":"city_clean",
    "zip_right":"zip"
},inplace=True)

/tmp/ipykernel_10522/2715793567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_df_2.rename(columns={


In [50]:
matched_df_2.columns,len(matched_df_2.columns)

(Index(['loan_amount', 'business_name', 'address', 'city', 'state', 'zip',
        'naics_code', 'business_type', 'race_ethnicity', 'gender', 'veteran',
        'non_profit', 'jobs_reported', 'date_approved', 'lender', 'cd',
        'source_file', 'na_flag', 'dupe_flag', 'year_approved', 'address_clean',
        'state_clean', 'city_clean'],
       dtype='object'),
 23)

In [51]:
refined_matched_df_1.to_csv("data/output/matched_df_1.csv")

In [52]:
matched_df_2.to_csv("data/output/matched_df_2.csv")